In [1]:
import os
from time import time, localtime
import logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import numpy as np
# from utils import plot_graph
# import tensorflow_federated as tff
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras import losses, metrics, optimizers
import random
import pandas as pd
import sys
import matplotlib.pyplot as plt
import nest_asyncio
from pathlib import Path
# from tensorflow_federated.python.research.utils import checkpoint_manager
# import tensorflow_addons as tfa
nest_asyncio.apply()
SEED = 1337
tf.random.set_seed(SEED)

In [2]:
def most_frequent(List):
    return max(set(List), key=List.count)

In [12]:
test_dir = "/home/jovyan/dataset/GQUIC_small/Test/GQUIC_test_10.feather"
test = pd.read_feather(test_dir)
print(test)
test.to_csv("/home/jovyan/dataset/GQUIC_small/Test/GQUIC_test_10.csv")


       flow_id    0    1    2    3    4    5    6    7    8    9  Label
0           29   16   10  149  182  192  172  158  109   13   79      1
1           29   16   10  168  155  186  253  154  217  174   83      1
2           29   16   10  167  101   83  237  126   84  226   69      1
3           29   16   10  166  230  173   64    4  169   36  232      1
4           29   16   10  165   59   89  192  140  124  161  173      1
...        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...    ...
55595   138365  144  111   86  255  137  119   88  101  151   95      2
55596   138365  144  239   86  254  137  119   84  165  151   95      2
55597   138365  144  111   87   16  137  119  152   37  151   95      2
55598   138365  144  111   87    6  137  119  114  165  151   95      2
55599   138365  144  111   87   17  137  119  155  229  151   95      2

[55600 rows x 12 columns]


In [19]:
result = test.groupby('flow_id')['Label'].apply(list).to_dict()
# print(result, type(result))
flow_label = []
for flow in result:
    flow_label.append(most_frequent(result[flow]))

flow_label = np.array(flow_label)

In [20]:
true_test = test.drop('flow_id', axis=1)

In [21]:
x_test = true_test.drop('Label', axis=1).to_numpy()/255
print(type(x_test))

<class 'numpy.ndarray'>


In [ ]:
x_test[20]

In [24]:
x_test = x_test.reshape(-1,20,512,1)

In [ ]:
x_test[1]

### Model Definition

In [25]:
model = tf.keras.models.load_model("/home/jovyan/dataset/model.h5")

### Predict

#### Predict all

In [27]:
predictions = model.predict(x_test,verbose=2,use_multiprocessing=True,batch_size=64)

ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/opt/conda/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "seq